In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

learning_rate = 0.001
total_epoch = 30
batch_size = 128

n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

# n_step 차원 추가
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

W0725 16:41:59.748237 12560 deprecation.py:323] From <ipython-input-1-65a0b2379fef>:4: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0725 16:41:59.779467 12560 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0725 16:41:59.779467 12560 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0725 16:42:02.905244 12560 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0725 16:42:02.920907 12560 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz


W0725 16:42:04.488213 12560 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
# BasicLSTMCell, GRUCell 등 다양한 방식의 셀 제공, 직접 구현하려면 다름 신경망 보다 복잡한 계산식,
# 저수준부터 하려면 다른 신경망 보다 복잡한 계산 필요
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

W0725 16:42:28.288455 12560 deprecation.py:323] From <ipython-input-2-e006f918b220>:1: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [4]:
# 셀, 입력값, 자료형 만으로 간단히 신경망 생성 가능
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

W0725 17:07:01.154652 12560 deprecation.py:323] From <ipython-input-4-d10edd0681b7>:2: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0725 17:07:01.237895 12560 deprecation.py:506] From c:\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0725 17:07:01.252842 12560 deprecation.py:506] From c:\python36\lib\site-packages\tensorflow\python\ops\rnn_cell_impl.py:459: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead 

In [ ]:
states = tf.zeros(batch_size)
# 한단계 학습 뒤 상태 저장, 그 상태를 다음 단계의 입력 상태로 하여 다시 학습, 반복하는게 이것
for i in range(n_step):
    outputs, states = cell(X[[:i]], states)

In [5]:
# 최종 출력값을 만들어 보자, 원핫 인코딩이므로 손실함수는 tf.nn.softmax_cross_entropy_with_logits_v2사용
# RNN 출력 값은 각 단계가 포함된 [batch_size, n_step, n_hidden] 형태로 출력
# dynamic_rnn 함수 옵션중 time_major를 True로 하면 [n_step, batch_size, n_hidde]형태로 출력
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

In [6]:
model = tf.matmul(outputs, W) + b

In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [8]:
# 신경망 학습하고 결과 확인하는 코드 작성 할 것, 앞장 코드와 비슷 입력따라 데이터 형태 바껴
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(total_epoch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys})
        
        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
    
print('최적화 완료!')

is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy, feed_dict={X:test_xs, Y:test_ys}))

Epoch: 0001 Avg. cost = 0.534
Epoch: 0002 Avg. cost = 0.224
Epoch: 0003 Avg. cost = 0.176
Epoch: 0004 Avg. cost = 0.156
Epoch: 0005 Avg. cost = 0.135
Epoch: 0006 Avg. cost = 0.123
Epoch: 0007 Avg. cost = 0.112
Epoch: 0008 Avg. cost = 0.111
Epoch: 0009 Avg. cost = 0.117
Epoch: 0010 Avg. cost = 0.103
Epoch: 0011 Avg. cost = 0.094
Epoch: 0012 Avg. cost = 0.093
Epoch: 0013 Avg. cost = 0.089
Epoch: 0014 Avg. cost = 0.091
Epoch: 0015 Avg. cost = 0.080
Epoch: 0016 Avg. cost = 0.085
Epoch: 0017 Avg. cost = 0.083
Epoch: 0018 Avg. cost = 0.074
Epoch: 0019 Avg. cost = 0.076
Epoch: 0020 Avg. cost = 0.078
Epoch: 0021 Avg. cost = 0.077
Epoch: 0022 Avg. cost = 0.076
Epoch: 0023 Avg. cost = 0.071
Epoch: 0024 Avg. cost = 0.072
Epoch: 0025 Avg. cost = 0.073
Epoch: 0026 Avg. cost = 0.066
Epoch: 0027 Avg. cost = 0.075
Epoch: 0028 Avg. cost = 0.065
Epoch: 0029 Avg. cost = 0.057
Epoch: 0030 Avg. cost = 0.061
최적화 완료!
정확도: 0.973
